# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [6]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [10]:
import pandas as pd

# Table and definitions sample
data = {
    'table': ['Employee', 'Department'],  # Example table names
    'definition': [
        'Contains information about employees such as name, hire_date, and salary.',
        'Contains department details like department name and manager.'
    ]  # Corresponding table definitions
}

df = pd.DataFrame(data)
print(df)


        table                                         definition
0    Employee  Contains information about employees such as n...
1  Department  Contains department details like department na...


In [11]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [12]:
print(text_tables)

Employee: Contains information about employees such as name, hire_date, and salary.
Department: Contains department details like department name and manager.


In [13]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [18]:
pqt1 = prompt_question_tables.format(
    tables=text_tables,
    question="What is the total salary paid in each department?"
)


In [19]:
print(return_OAI(pqt1))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [26]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question=#ENTER YOUR QUERY HERE)

In [ ]:
print(return_OAI(pqt3))

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
import openai

openai.api_key = "OPENAI_API_KEY"

# Refined prompt with schema details
prompt = """
Generate an SQL query based on the following details:
- The `employees` table contains columns: `id`, `name`, `hire_date`, `salary`, `department_id`.
- The `departments` table contains columns: `department_id` and `department_name`.
- Find the names of employees and their departments where the salary is greater than $80,000.
"""

# Call the OpenAI API
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that generates SQL queries."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=150,
    temperature=0.7
)

# Extract and print the SQL query
sql_query = response['choices'][0]['message']['content'].strip()
print(f"Refined Prompt:\n{prompt}")
print(f"Generated SQL Query:\n{sql_query}")


In [ ]:
import openai

openai.api_key = "OPENAI_API_KEY"

problematic_prompt = """
Generate an SQL query to find employee names and their managers.
"""

response_problematic = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that generates SQL queries."},
        {"role": "user", "content": problematic_prompt}
    ],
    max_tokens=150,
    temperature=0.7
)

print(f"Problematic Prompt:\n{problematic_prompt}")
print(f"Generated SQL Query:\n{response_problematic['choices'][0]['message']['content'].strip()}\n")

fixed_prompt = """
Generate an SQL query based on the following schema:
- The `employees` table contains: `id`, `name`, `manager_id`, `department_id`.
- The `departments` table contains: `department_id`, `department_name`.
Find employee names and their managers' names using the `manager_id` column.
"""

response_fixed = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that generates SQL queries."},
        {"role": "user", "content": fixed_prompt}
    ],
    max_tokens=150,
    temperature=0.7
)

print(f"Fixed Prompt:\n{fixed_prompt}")
print(f"Generated SQL Query:\n{response_fixed['choices'][0]['message']['content'].strip()}")